In [1]:
import cv2
import numpy as np

def video_to_frames(video_path):
    # Read the video
    video = cv2.VideoCapture(video_path)

    frames = []

    while video.isOpened():
        ret, frame = video.read()

        if not ret:
            break

        # Convert the frame to grayscale if needed
        # gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Add the frame to the list
        frames.append(frame)

    video.release()

    return frames

# Example usage
video_path = "dots_quasi.wmv"
frames = video_to_frames(video_path)
print("Number of frames:", len(frames))

Number of frames: 390


In [2]:
from PIL import Image
import cv2
import glob
from scipy import interpolate
from scipy.interpolate import CubicSpline
from matplotlib import pyplot as plt


In [3]:
def loadVideo(filepath):
    vid = cv2.VideoCapture(filepath)
    _, image = vid.read()
    count = 0
    success = True
    n_row, n_col, _ = image.shape
    data = []
    while success:
        data.append(image.flatten()) 
        success,image = vid.read()
        count += 1
    vid.release() 
    data = np.array(data)
    print(count, " frames extracted")
    print("frame size = ", (n_row , n_col))
    print("data shape =", data.shape)
    return data , (n_row, n_col)

data , shape = loadVideo('dots_quasi.wmv')
n_row, n_col = shape

390  frames extracted
frame size =  (100, 200)
data shape = (390, 60000)


In [4]:
def loadFrame(framefilepath): 
    image_path = framefilepath
    image = cv2.imread(image_path)
    blue_channel, green_channel, red_channel = cv2.split(image)
    
    pixels = image.reshape(-1, 3)

    return pixels 

In [5]:
import os
from pathlib import Path

folder_path = "/Users/acrv/Documents/GitHub/MSRI-UP2023/frames"

#Get a list of file names in the folder
file_names = os.listdir(folder_path)

#Define the custom sorting key function
def get_file_number(file_path):
    # Extract the number from the file name
    try:
        file_name = os.path.basename(file_path)  # Extract the file name from the full path
        number = int(file_name.split('.')[0].replace('frame_', ''))  # Assuming the number is before the file extension
    except ValueError:
        number = float('inf')  # Assign a high value if the file name doesn't contain a number
    return number
#Sort the file names based on the custom sorting key

sorted_file_names = sorted(file_names, key=get_file_number)

filepath_list=[]

#Iterate over the sorted file names
for file_name in sorted_file_names:
    file_path = os.path.join(folder_path, file_name)
    filepath_list.append(file_path)
    
filepath_array=np.array(filepath_list)
filepath2=[]

for i in range(len(filepath_array)-1):
    filepath2.append(filepath_array[i])
dummy_matrix=[]    
for path in np.array(filepath2): 
    pixels=loadFrame(path)
    dummy_matrix.append(pixels)
    
print(len(dummy_matrix[0]))

20000


In [6]:
import os
import cv2
import numpy as np

# Folder path containing the images
folder_path = '/Users/acrv/Documents/GitHub/MSRI-UP2023/frames'  # Replace with the actual folder path

# Create an empty array to store all RGB values
rgb_array = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Get the full file path
        file_path = os.path.join(folder_path, filename)

        # Read the image using OpenCV
        image = cv2.imread(file_path)

        # Convert the image to RGB format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get the dimensions of the image
        height, width, _ = image.shape

        # Reshape the image array to a 2D array of shape (height * width, 3)
        reshaped_image = np.reshape(image, (height * width, 3))

        # Append the RGB values of the current image to the array
        rgb_array.append(reshaped_image)

# Convert the RGB values array to a NumPy array
rgb_array = np.array(rgb_array)

#print(type(rgb_array))

In [84]:
rgb_array.shape

(390, 20000, 3)

In [8]:
#This cell contains rgb array that has 100 by 200 (wrong wording but itll do the trick for now)

import os
import numpy as np
from PIL import Image

# Folder path containing the images
folder_path = '/Users/acrv/Documents/GitHub/MSRI-UP2023/frames'  # Replace with the actual folder path

# Create an empty array to store all RGB values
rgb = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Get the full file path
        file_path = os.path.join(folder_path, filename)

        # Open the image file
        image = Image.open(file_path)

        # Convert the image to RGB mode (if not already in RGB)
        image = image.convert('RGB')

        # Get the dimensions of the image
        width, height = image.size

        # Create an empty NumPy array to store RGB values for the current image
        image_rgb_values = np.empty((height, width, 3), dtype=np.uint8)

        # Iterate over each pixel in the image
        for y in range(height):
            for x in range(width):
                # Get the RGB values of the pixel at (x, y)
                r, g, b = image.getpixel((x, y))

                # Assign the RGB values to the corresponding position in the NumPy array
                image_rgb_values[y, x] = [r, g, b]

        # Append the RGB values of the current image as a single entry in the array
        rgb.append(image_rgb_values)

rgb = np.array(rgb)
        
#print(rgb[0][50])


In [3]:


def getSlidingWindowVideo(I, dim, Tau, dT):
    N = I.shape[0]  # Number of frames
    H = I.shape[1]  # Height
    W = I.shape[2]  # Width
    C = I.shape[3]  # Color channels

    P = H * W * C  # Number of pixels (possibly after PCA)
    pix = np.arange(P)

    NWindows = int(np.floor((N - dim * Tau) / dT))
    X = np.zeros((NWindows, dim * P))
    idx = np.arange(N)

    for i in range(NWindows):
        idxx = dT * i + Tau * np.arange(dim)
        start = int(np.floor(idxx[0]))
        end = int(np.ceil(idxx[-1]))
        t_values = idx[start:end + 1]
        x_values = pix
        z_values = I[t_values, :, :, :].reshape(-1, H * W * C)
        
        # Perform interpolation
        f = interpolate.interp2d(x_values, t_values, z_values, kind='linear')
        interpolated_values = f(pix, idxx)

        X[i, :] = interpolated_values.flatten()

    return X

wSize = 16
dim = 5
Tau = wSize/float(dim)

data = rgb
desiredSamples = 600
M = data.shape[0] - wSize + 1
dT = M/float(desiredSamples)

Z = getSlidingWindowVideo(data, dim, Tau, dT)

#print(Z[50][70])

NameError: name 'rgb' is not defined

In [87]:
# Auxiliary video functions
def getSlidingWindowVideo(I, dim, Tau, dT):
    N = I.shape[0] #Number of frames
    P = I.shape[1] #Number of pixels (possibly after PCA)
    pix = np.arange(P)
    NWindows = int(np.floor((N-dim*Tau)/dT))
    X = np.zeros((NWindows, dim*P))
    idx = np.arange(N)
    for i in range(NWindows):
        idxx = dT*i + Tau*np.arange(dim)
        start = int(np.floor(idxx[0]))
        end = int(np.ceil(idxx[-1]))
        
        # Create grid points for each dimension
        grid_x, grid_y = np.meshgrid(pix, idx[start:end + 1], indexing='ij')

        # Reshape the input array
        values = I[start:end + 1, :, :].reshape((end - start + 1, -1)).T

        # Create RegularGridInterpolator object
        f = interpolate.RegularGridInterpolator((idx[start:end + 1], pix), values, method='linear')

        # Evaluate the interpolated values on the grid points
        X[i, :] = f(np.column_stack((grid_y.flatten(), grid_x.flatten())))

    return X

wSize = 16
dim = 5
Tau = wSize/float(dim)

data = rgb_array

desiredSamples = 600
M = data.shape[0] - wSize + 1
dT = M/float(desiredSamples)

X = getSlidingWindowVideo(data, dim, Tau, dT)

#XS = X - np.mean(X, 1)[:, None]
#XS = XS/np.sqrt(np.sum(XS**2, 1))[:, None]


ValueError: There are 14 points and 60000 values in dimension 0

# Auxiliary video functions
def getSlidingWindowVideo(I, dim, Tau, dT):
    N = I.shape[0] #Number of frames
    P = I.shape[1] #Number of pixels (possibly after PCA)
    pix = np.arange(P)
    NWindows = int(np.floor((N-dim*Tau)/dT))
    X = np.zeros((NWindows, dim*P))
    idx = np.arange(N)
    for i in range(NWindows):
        idxx = dT*i + Tau*np.arange(dim)
        start = int(np.floor(idxx[0]))
        end = int(np.ceil(idxx[-1]))
        f = interpolate.interp2d(pix, idx[start:end+1], I[idx[start:end+1], :], kind='linear')
        X[i, :] = f(pix, idxx).flatten()
    return X

wSize = 16
dim = 5
Tau = wSize/float(dim)

data = np.array(dummy_matrix)
desiredSamples = 600
M = data.shape[0] - wSize + 1
dT = M/float(desiredSamples)

Z = getSlidingWindowVideo(data, dim, Tau, dT)


In [67]:
rgb_array.shape

(390, 20000, 3)

In [64]:
# Persistence Computation
from ripser import ripser 
from persim import plot_diagrams

rips_persistence = ripser(X.T, maxdim=1)

#dgms = rips_persistence['dgms']

#plt.figure(figsize = (5,5)) 
#plot_diagrams(dgms, title='Persistence Diagrams')

In [1]:
#topological data analysis
from persim import plot_diagrams
from dreimac import CircularCoords 
from dreimac import CircleMapUtils as CMU

#plotting and visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure(data=[go.Scatter3d(
    x=Z[0], y=Z[1], z=Z[2], 
    mode ='markers', 
    #marker= dict(size = 3, color = np.arange(data.shape[0]) , colorscale='plasma' , opacity = 1)
    marker=dict(size = 2, color = 'grey', opacity = 1)
)])

fig.update_layout( width=900, height=450) 
fig.show()

NameError: name 'Z' is not defined

In [ ]:
coho_classes = [0, 1]

circular_coords = []

n_lands = 200


cc = CircularCoords(Z, n_landmarks=n_lands)

XS = Z - np.mean(Z, 1)[:, None]
XS = XS/np.sqrt(np.sum(XS**2, 1))[:, None]

for i in coho_classes:
    theta = cc.get_coordinates(perc= 0, cocycle_idx=i)
    theta = CMU.to_sinebow(np.pi + CMU.center(theta))
    circular_coords.append(theta)
    
fig = make_subplots(rows=1, cols=len(coho_classes), 
                    subplot_titles = tuple('Circ Coord '+str(i+1) for i in coho_classes),
                    specs = [[{'type': 'scatter3d'} for i in coho_classes]])

for i, coord in enumerate(circular_coords):
    fig.add_trace(
        go.Scatter3d(x=XS[0], y=XS[1], z=XS[2],   mode ='markers', 
        marker=dict(size = 2 , color =  circular_coords[i])),  
        row=1, col=i+1)
    fig.update_scenes(xaxis=dict( ticks='', showticklabels=False), 
                      yaxis=dict( ticks='', showticklabels=False), 
                      zaxis=dict( ticks='', showticklabels=False), 
                      aspectmode='data', row=1, col=i+1)

fig.update_layout( showlegend=False )
fig.show()

In [ ]:
coho_classes = [0, 1]

circular_coords = []

n_lands = 200

cc = CircularCoords(Z.T, n_landmarks=n_lands)

for i in coho_classes:
    theta = cc.get_coordinates(perc= 0, cocycle_idx=i)
    theta = CMU.to_sinebow(np.pi + CMU.center(theta))
    circular_coords.append(theta)
    
fig = make_subplots(rows=1, cols=len(coho_classes), 
                    subplot_titles = tuple('Circ Coord '+str(i+1) for i in coho_classes),
                    specs = [[{'type': 'scatter3d'} for i in coho_classes]])

In [ ]:
# Reshape the image array to a 2D array of shape (num_images * num_pixels, 3)
#reshaped_array = rgb.reshape(-1, 3)
#print(len(rgb))

# Extract the red values using array indexing
red_values = rgb[:, 0]  # Index 0 corresponds to the red channel

# Print the array of red values
print(red_values)
